In [2]:
import pickle as pkl
import numpy as np
from tqdm import tqdm
import os
import json

In [3]:
in_dir = "data/pkl_fine"
in_file = "jihualde_CV.pkl"
out_dir = "data/json/"
out_file = "data_fine.json"

In [4]:
in_path = f'{in_dir}/{in_file}'
out_path = f'{out_dir}/{out_file}'

In [5]:
def get_record(in_path):
    with open(in_path, "rb") as f_in, open(out_path, "w") as f_out:
        data = pkl.load(f_in)
        json_format = data.json_format
        for record in tqdm(data.record):
            # print(record)
            from_idx = record['from']
            to_idx = record['to']
            type_str = record['type']
            buf_string = "$ROOT" if from_idx == -1 else json_format[from_idx]['text']
            stk_string = "$ROOT" if to_idx == -1 else json_format[to_idx]['text']
            yield {
                'buf_str': buf_string,
                'stk_str': stk_string,
                'type': type_str
            }
            # print(f'{buf_string},{stk_string},{type_str}')
    

In [1]:
# fix a bug where the discard function had the from and to indices both point to themselves 

def fix_discard(in_path):
    
    data = None
    print(in_path)
    with open(in_path, "rb") as f_in:
        data = pkl.load(f_in)

    stack = [-1] 

    for entry in data.record:
        print("ORIGINAL", entry)
        if(entry['type'] == 'pop'):
            stack.pop()
        if(entry['type'] == 'merge'):
            stack.pop()
            stack.append(entry['from'])
        if(entry['type'] == 'subordinate'):
            stack.append(entry['from'])
        if(entry['type'] == 'discard'):
            entry['to'] = stack[-1]
        print("NEW", entry)

    with open(in_path, "wb") as f_out:
        pkl.dump(data, f_out)
        # above line is pretty dangerous, should wait until everything is ready

In [137]:
def pkl_to_json(in_dir = "data/pkl_fine", out_dir = "data/json/", out_name = "data_fine.json"):
    to_write = {
        'buf_str': [],
        'stk_str': [],
        'type': []
    }

    for file in os.listdir(in_dir):
        full_path = f'{in_dir}/{file}'
        print(full_path)
        if(os.path.isfile(full_path)):
            for entry in get_record(full_path):
                for key, val in entry.items():
                    to_write[key].append(val)
                    # print(key, val)
    # print(to_write)
    print(f'Found {len(to_write['buf_str'])} elements')
    out_path = f'{out_dir}/{out_name}'  

    with open(out_path, "w") as f_out:
        json.dump(to_write, f_out)

In [138]:
pkl_to_json()
pkl_to_json(in_dir = "data/pkl_coarse", out_name = "data_coarse.json")

data/pkl_fine/BhattCV 221.pkl


100%|██████████| 1310/1310 [00:00<00:00, 139820.81it/s]


buf_str RAKESH MOHAN BHATT
stk_str $ROOT
type subordinate
buf_str Department of Linguistics
stk_str RAKESH MOHAN BHATT
type pop
buf_str Department of Linguistics
stk_str $ROOT
type subordinate
buf_str University of Illinois
stk_str Department of Linguistics
type merge
buf_str Urbana, IL 61801
stk_str University of Illinois
type merge
buf_str rbhatt@illinois.edu
stk_str Urbana, IL 61801
type merge
buf_str ADMINISTRATIVE POSTIONS
stk_str rbhatt@illinois.edu
type pop
buf_str ADMINISTRATIVE POSTIONS
stk_str $ROOT
type subordinate
buf_str Acting Head, Department of Linguistics, University of Illinois, Jan 1-May 15, 2016.
stk_str ADMINISTRATIVE POSTIONS
type subordinate
buf_str Associate Head, Department of Linguistics, University of Illinois, Fall 2010-to date.
stk_str Acting Head, Department of Linguistics, University of Illinois, Jan 1-May 15, 2016.
type pop
buf_str Associate Head, Department of Linguistics, University of Illinois, Fall 2010-to date.
stk_str ADMINISTRATIVE POSTIONS
type s

100%|██████████| 206/206 [00:00<00:00, 137060.06it/s]


buf_str Qizheng He
stk_str $ROOT
type subordinate
buf_str University of Illinois at Urbana-Champaign
stk_str Qizheng He
type pop
buf_str University of Illinois at Urbana-Champaign
stk_str $ROOT
type subordinate
buf_str Computer Science Department
stk_str University of Illinois at Urbana-Champaign
type merge
buf_str Address: 3305 Thomas Siebel Center, Urbana, IL 61801
stk_str Computer Science Department
type merge
buf_str Phone: (217) 693-8314
stk_str Address: 3305 Thomas Siebel Center, Urbana, IL 61801
type merge
buf_str Email: qizheng6@illinois.edu
stk_str Phone: (217) 693-8314
type merge
buf_str https://sites.google.com/site/qizhenghe96/
stk_str Email: qizheng6@illinois.edu
type merge
buf_str Research Interests
stk_str https://sites.google.com/site/qizhenghe96/
type pop
buf_str Research Interests
stk_str $ROOT
type subordinate
buf_str Computational Geometry, Algorithms and Data Structures, and Fine-Grained Complexity.
stk_str Research Interests
type subordinate
buf_str Education
stk_

...

data/pkl_coarse/cv-Qizheng2022.pkl


100%|██████████| 164/164 [00:00<00:00, 138766.56it/s]


buf_str Qizheng He
stk_str $ROOT
type subordinate
buf_str University of Illinois at Urbana-Champaign
stk_str Qizheng He
type pop
buf_str University of Illinois at Urbana-Champaign
stk_str $ROOT
type subordinate
buf_str Computer Science Department
stk_str University of Illinois at Urbana-Champaign
type merge
buf_str Address: 3305 Thomas Siebel Center, Urbana, IL 61801
stk_str Computer Science Department
type merge
buf_str Phone: (217) 693-8314
stk_str Address: 3305 Thomas Siebel Center, Urbana, IL 61801
type merge
buf_str Email: qizheng6@illinois.edu
stk_str Phone: (217) 693-8314
type merge
buf_str https://sites.google.com/site/qizhenghe96/
stk_str Email: qizheng6@illinois.edu
type merge
buf_str Research Interests
stk_str https://sites.google.com/site/qizhenghe96/
type pop
buf_str Research Interests
stk_str $ROOT
type subordinate
buf_str Computational Geometry, Algorithms and Data Structures, and Fine-Grained Complexity.
stk_str Research Interests
type subordinate
buf_str Education
stk_

100%|██████████| 565/565 [00:00<00:00, 135292.40it/s]

buf_str Ryan K. Shosted
stk_str $ROOT
type subordinate
buf_str Professor
stk_str Ryan K. Shosted
type pop
buf_str Professor
stk_str $ROOT
type subordinate
buf_str University of Illinois at Urbana-Champaign
stk_str Professor
type merge
buf_str Appointments:
stk_str University of Illinois at Urbana-Champaign
type pop
buf_str Appointments:
stk_str $ROOT
type subordinate
buf_str College of Liberal Arts & Sciences
stk_str Appointments:
type subordinate
buf_str Department of Linguistics (100%)
stk_str College of Liberal Arts & Sciences
type merge
buf_str Department of Spanish and Portuguese (0%)
stk_str Department of Linguistics (100%)
type merge
buf_str Program in American Indian Studies (0%)
stk_str Department of Spanish and Portuguese (0%)
type merge
buf_str Program in Translation and Interpreting Studies (0%)
stk_str Program in American Indian Studies (0%)
type merge
buf_str Beckman Institute for Advanced Science and Technology
stk_str Program in Translation and Interpreting Studies (0%)